In [7]:
# GAUSS & CONULAR Combination
# Created: 10/24/2022
# Author: Cameron King
# Function Authors: Alec Church

# Description:
#
# This function will combine the gaussian distribution output from Alec's function, with the conular dispursion function.
# This will be used for the combined assumption that the beam has a gaussian distribution, that is aimed perfectly to dispurse in a conular method.
# The laser within the sc creates a gaussian beam, and a lens is used to refract the beam into a cone coming from the lens as an approximate point source.

# Functions:
# This function needs access to the transmission_funcs.ipynb file to call gaussian_transmission_func


# Errors & Checks:
# This function also gives a check to if gaussian_transmission_func is running correctly, as in if it is outputting the correct total shell percentage.
# This means that this function throws an error if it finds that the shell percentages dont sum close to 100%. 
# It will never be perfect, but it seems to usually be about 99.02% with current values, and it should always, if the function runs right with given values, return
# something close to 100%. This efficeincy loss can be calculated, but I think we should include it as an uncertainty of our gaussian model.


# libraries
import math
import numpy
import mpmath

# functions
from ipynb.fs.full.transmission_funcs import *

In [8]:
def gaussNcone_transmission_func(r_aperture, r_ave, d_ave, d, P0):
    
    r = r_aperture/1.52 # 99% of the beam is in 1.52*r, where r is the radius of the beam IN THE GAUSSIAN FORMULATION
                        # dividing by 1.52 here allows for the calculation of a beam that contains 99% in r_aperature = r*1.52
        
    d_lens = 1 # hardcoded value for distance from the laser to the output lens
    
    # Get the flux distribution and radii for gaussian beam with radius of the output lens, at the output lens
    F_disp = gaussian_transmission_func(r, d_lens, d_lens, P0) # this does the gaussian formulation of the math to determine power in each shell
    
    # determine cone shape and radius ratio
    alpha = mpmath.atan((r_ave-r_aperture)/d_ave) # view angle, half of FOV basically
    
    r_actual = d*mpmath.tan(alpha) # actual radius of beam at actual distance d
    r_actual = float(r_actual)
    
    r_ratio = r_actual/r_ave # multiply this by the radius at the output lense to get F_disp radii scaled as a cone
    
    for i in range(0,len(F_disp[0])-1): # Calculates the scaled radii of the larger 'spotlight'
        F_disp[0][i] = F_disp[0][i]*r_ratio
    
    
    # CHECK: is the sum of the shell percentages close to 100%? throw error if not
    tot_shell_perc = sum(F_disp[1][:])-F_disp[1][0]    
    if tot_shell_perc < 0.98:
        print('Total shell percentage = ',tot_shell_perc)
        raise ValueError('Total Shell Percentage doesnt sum to >98%. Check gaussian_transmission_func & calls.')
        
        
        
    return F_disp

    